<a href="https://colab.research.google.com/github/arjunKumbakkara/MachineLearning_ArtificialIntelligence/blob/master/DGA_Malware_Detection_using_Bigram_LSTM_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tldextract

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

"""Run experiments and create figs"""
import itertools
import os
import pickle
import matplotlib
matplotlib.use('Agg')
import numpy as np

from numpy import interp
from sklearn.metrics import roc_curve, auc

In [0]:
"""Train and test bigram classifier"""
#import dga_classifier.data as data
from keras.layers.core import Dense
from keras.models import Sequential
import sklearn
from sklearn import feature_extraction

from sklearn.model_selection import train_test_split
#****************************************************************************************************
"""Generates data for train/test algorithms"""
from datetime import datetime
try:
    from StringIO import StringIO ## for Python 2
except ImportError:
    from io import StringIO ## for Python 3
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

from zipfile import ZipFile

import _pickle as cPickle
import os
import random
import tldextract
import pandas as pd

Using TensorFlow backend.


In [0]:
def domain_extract(uri):
    ext = tldextract.extract(uri)
    if (not ext.suffix):
        return np.nan
    else:
        return ext.domain

In [0]:
import random
def get_alexa(size_of_data):
    alexa_dataframe = pd.read_csv('/content/drive/My Drive/athulColab/data/alexa_100k.csv', names=['rank','uri'], header=None, encoding='utf-8')
    alexa_dataframe.head()
    alexa_dataframe['domain'] = [ domain_extract(uri) for uri in alexa_dataframe['uri']]
    del alexa_dataframe['rank']
    del alexa_dataframe['uri']
    alexa_dataframe.head()
    alexa_dataframe = alexa_dataframe.dropna()
    alexa_dataframe = alexa_dataframe.drop_duplicates()

    alexa_dataframe = alexa_dataframe[:size_of_data]
    #test_data = data[50%:]
    #print("len of each data",len(alexa_dataframe))
    labels = []
    domains = list(alexa_dataframe['domain'])
    labels += ['benign']*(len(domains))
    #print("in_get_alexa",domains, labels)
    return domains, labels


In [0]:
# Original code: https://github.com/baderj/domain_generation_algorithms/blob/master/banjori/dga.py
def map_to_lowercase_letter(s):
    return ord('a') + ((s - ord('a')) % 26)

def next_domain(domain):
    dl = [ord(x) for x in list(domain)]
    dl[0] = map_to_lowercase_letter(dl[0] + dl[3])
    dl[1] = map_to_lowercase_letter(dl[0] + 2*dl[1])
    dl[2] = map_to_lowercase_letter(dl[0] + dl[2] - 1)
    dl[3] = map_to_lowercase_letter(dl[1] + dl[2] + dl[3])
    return ''.join([chr(x) for x in dl])

def generate_banjori(nr_domains, seed='hereisaseeddomainthatweuse.com'):
    ret = []

    for i in range(nr_domains):
        seed = next_domain(seed)

        ret.append(seed)

    return ret

In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/simda/dga.py
def generate_simda(nr_domains, length=10, tld="com", key = "1676d5775e05c50b46baa5579d4fc7", base = 0x45AE94B2):
    consonants = "qwrtpsdfghjklzxcvbnmv"
    vowels = "eyuioa"

    step = 0
    for m in key:
        step += ord(m)

    ret = []

    for nr in range(nr_domains):
        domain = ""
        base += step

        for i in range(length):
            index = int(base/(3+2*i))
            if i % 2 == 0:
                char = consonants[index % 20]
            else:
                char = vowels[index % 6]
            domain += char

        if tld:
            domain += "." + tld
        
        ret.append(domain)

    return ret

In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/ramnit/dga.py
class RandInt:

    def __init__(self, seed): 
        self.seed = seed

    def rand_int_modulus(self, modulus):
        ix = self.seed                
        ix = 16807*(ix % 127773) - 2836*(ix / 127773) & 0xFFFFFFFF        
        self.seed = ix 
        return ix % modulus 

def get_domains(seed, nr, add_tld=False):
    r = RandInt(seed)

    for i in range(nr):
        seed_a = r.seed
        domain_len = r.rand_int_modulus(12) + 8
        seed_b = r.seed
        domain = ""
        for i in range(domain_len):
            char = chr(ord('a') + r.rand_int_modulus(25))
            domain += char

        if add_tld:
            domain += ".com"
            
        m = seed_a*seed_b
        r.seed = (m + m//(2**32)) % 2**32 
        yield domain

def generate_ramnit(nr, seed, add_tld=False):
    return [x for x in get_domains(seed, nr, add_tld)]

#import ramnit
#domains = ramnit.generate_domains(0x123abc12, 10)

In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/qakbot/dga.py
import binascii
from datetime import datetime
from ctypes import c_uint
import argparse

def date_to_seed(date, seed):
    dx = (date.day-1) // 10 
    data = "{}.{}.{}.{:08x}".format(
            dx if dx <= 2 else 2,
            date.strftime("%b").lower(), 
            date.year, 
            seed)
    crc = c_uint(binascii.crc32(data.encode('ascii')))
    return crc

def _int32(x):
    return int(0xFFFFFFFF & x)

class MT19937:

    def __init__(self, seed):
        # Initialize the index to 0
        self.index = 624
        self.mt = [0] * 624
        self.mt[0] = seed  # Initialize the initial state to the seed
        for i in range(1, 624):
            self.mt[i] = _int32(
                1812433253 * (self.mt[i - 1] ^ self.mt[i - 1] >> 30) + i)

    def extract_number(self):
        if self.index >= 624:
            self.twist()

        y = self.mt[self.index]
        y = y ^ y >> 11
        y = y ^ y << 7 & 2636928640
        y = y ^ y << 15 & 4022730752
        y = y ^ y >> 18

        self.index = self.index + 1
        return _int32(y)

    def twist(self):
        for i in range(0, 624):
            y = _int32((self.mt[i] & 0x80000000) +
                       (self.mt[(i + 1) % 624] & 0x7fffffff))
            self.mt[i] = self.mt[(i + 397) % 624] ^ y >> 1

            if y % 2 != 0:
                self.mt[i] = self.mt[i] ^ 0x9908b0df
        self.index = 0

    def rand_int(self, lower, upper):
        r = self.extract_number()
        r &= 0xFFFFFFF
        t = lower + float(r) / (2**28)*(upper - lower + 1) 
        t = int(t)
        return t
        

def generate_qakbot(nr_domains, date=None, 
                     tlds = ["com", "net", "org", "info", "biz", "org"], 
                     sandbox=False, seed=0):

    date = date if date else datetime.now()

    seed = date_to_seed(date, seed).value + sandbox
    mt = MT19937(seed)

    ret = []
    for i in range(nr_domains):
        if tlds:
            tld_nr = mt.rand_int(0, len(tlds) - 1)

        length = mt.rand_int(8, 25)
        domain = ""

        for l in range(length):
            domain += chr(mt.rand_int(0, 25) + ord('a'))

        if tlds:
            domain += "." + tlds[tld_nr]
        
        ret.append(domain)

    return ret
#import qakbot
#qakbot.generate_domains(10000)



In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/dircrypt/dga.py
import argparse

class RandInt:

    def __init__(self, seed): 
        self.seed = seed

    def rand_int_modulus(self, modulus):
        ix = self.seed                
        ix = int(16807*(ix % 127773) - 2836*(ix / 127773)) & 0xFFFFFFFF        
        self.seed = ix
        return ix % modulus 

def generate_dircrypt(nr, seed="1DBA8930", tld=''):
    r = RandInt(int(seed, 16))

    ret = []

    for i in range(nr):
        domain_len = int(r.rand_int_modulus(12+1)) + 8
        domain = ""
        for i in range(domain_len):
            char = chr(ord('a') + int(r.rand_int_modulus(25+1)))
            domain += char
        domain += tld
        
        ret.append(domain)

    return ret

#import dircyrpt
#dircyrpt.generate_domains(10000, "1DBA8930")


In [0]:
# http://resources.infosecinstitute.com/domain-generation-algorithm-dga/
# cryptolocker
from datetime import datetime, timedelta


def generate_cryptolocke(year, month, day, length=32, tld=''):
    """ Generates a domain by considering the current date. """
    domain = ""

    for i in range(length):
        year = ((year ^ 8 * year) >> 11) ^ ((year & 0xFFFFFFF0) << 17)
        month = ((month ^ 4 * month) >> 25) ^ 16 * (month & 0xFFFFFFF8)
        day = ((day ^ (day << 13)) >> 19) ^ ((day & 0xFFFFFFFE) << 12)
        domain += chr(((year ^ month ^ day) % 25) + 97)

    domain += tld

    return domain


def generate_cryptolocker(num_domains, seed_num=100, length=32):
    """ Generates a list of domains based on a seed integer. """
    domains = []
    start_date = datetime(2016, 1, 1) + timedelta(days=seed_num)
    for i in range(round(num_domains)):
            new_date = start_date + timedelta(days=i*2+1)
            domains.append(generate_cryptolocke(new_date.year, new_date.month, new_date.day, length))
    return domains

# import cryptolocker
# cryptolocker.generate_domains(10, 1)


In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/corebot/dga.py
from datetime import datetime

def init_rand_and_chars(year, month, day, nr_b, r):
    r = (r + year + ((nr_b << 16) + (month << 8) | day)) & 0xFFFFFFFF
    charset = [chr(x) for x in range(ord('a'), ord('z'))] +\
            [chr(x) for x in range(ord('0'), ord('9'))]
            
    return charset, r

def generate_cbot(charset, r, tld=''):
    len_l = 0xC
    len_u = 0x18
    r = (1664525*r + 1013904223) & 0xFFFFFFFF
    domain_len = len_l + r % (len_u - len_l)
    domain = ""
    for i in range(domain_len, 0, -1):
        r = ((1664525 * r) + 1013904223) & 0xFFFFFFFF
        domain += charset[r % len(charset)] 
    domain += tld
    return r, domain

def generate_corebot(nr_domains, seed='1DBA8930', d=None, nr_b=1):
    d = datetime.now()

    charset, r = init_rand_and_chars(d.year, d.month, d.day, 1, int(seed, 16)) 
    ret = []
    for _ in range(nr_domains):
        r, d = generate_cbot(charset, r)
        ret.append(d)
    return ret
#import corebot
#corebot.generate_domains("1DBA8930", 100000)


In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/kraken/v2/dga_v2.py
import random
import time


def rand(r):
    t = (1103515245 * r + 12435) & 0xFFFFFFFF
    return t


def crop(r):
    return (r // 256) % 32768


def generate_kraken1(index, date, seed_set, temp_file=True, tld_set_nr=1):
    tld_sets = {1: ["com", "net", "tv", "cc"],
                2: ["dyndns.org", "yi.org", "dynserv.com", "mooo.com"],
                3: None}

    seeds = {'a': {'ex': 24938314, 'nex': 24938315},
             'b': {'ex': 1600000, 'nex': 1600001}}
    tlds = tld_sets[tld_set_nr]

    domain_nr = int(index/2)
    if temp_file:
        r = 3*domain_nr + seeds[seed_set]['ex']
    else:
        r = 3*domain_nr + seeds[seed_set]['nex']

    discards = (int(time.mktime(date.timetuple())) - 1207000000) // 604800 + 2
    if domain_nr % 9 < 8:
        if domain_nr % 9 >= 6:
            discards -= 1
        for _ in range(discards):
            r = crop(rand(r))

    rands = 3*[0]
    for i in range(3):
        r = rand(r)
        rands[i] = crop(r)
    domain_length = (rands[0]*rands[1] + rands[2]) % 6 + 7
    domain = ""
    for i in range(domain_length):
        r = rand(r)
        ch = crop(r) % 26 + ord('a')
        domain += chr(ch)

    if tlds:
        domain += "." + tlds[domain_nr % 4]
    return domain


def generate_kraken(num_domains, date, seed, tld_set):
    domains = []
    for i in range(num_domains):
        domains.append(generate_kraken1(i*2, date, seed, random.sample(range(2), 1)[0], tld_set))

    return domains

# import kraken_v2
# from datetime import datetime
# kraken_v2.generate_domains(10, datetime(2016, 1, 1), 'a', 1)

In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/locky/dgav2.py
import argparse 
from datetime import datetime

config = {
1: {
    # md5: 81e85dcaf482aba2f8ea047145490493
    # sha256: 9afb127e733b01952f00a8174291d39f740b6df2c90d0422b4d6f2e2e6bc7d1a
    # sample: https://virustotal.com/en/file/9afb127e733b01952f00a8174291d39f740b6df2c90d0422b4d6f2e2e6bc7d1a/analysis/
    'seed': 7077,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
2: {
    # md5: 5fb8f8f75342ff68ed8c79cc375f0cd8
    # sha256: bc7c45b5a05f3f0deea162578e45d5fb64c9aa72a81395083509c0f78b6ae1de
    # sample: https://malwr.com/analysis/NzFlYzRkOWZhZDliNDZmMThkMzkzMjU2ZmE5ODUzMjE/
    'seed': 5566,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
3: {
    # md5: 19079496f6abfafd9a99d02098556a79
    # sha256: 5dd6188efe13268bb9ac20ecdb257085e7d62163
    # sample: https://malwr.com/analysis/Yzg5ZWMyZDZmNGFhNGU0YWJjMzY3YjBmMjY4Y2JlMDM/
    'seed': 9106,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
4: {
    # md5: 0e223d578eaddec361498591ec8c1a19
    # sha256: 
    # sample: 
    'seed': 5579,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
5: {
    # md5: 6cb11f4066f74556dd14d27008d867b4
    # sha256: 353ea18baa6c9c796a7b48bcccbf4c9c3c6aa63f8b4dd55d796c65e22028b77b
    # sample: https://malwr.com/analysis/N2Q4YWUyM2I3Y2VlNGQwYzllMjczNzE2Njc1ZTFhZWI/
    'seed': 111,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
6: {
    # md5: ed534695e79a2a70e8b15f8873cdfa02
    # sha256:  
    # sample: https://www.sophos.com/en-us/threat-center/threat-analyses/viruses-and-spyware/Troj~Ransom-CUL/detailed-analysis.aspx
    'seed': 9044,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
7: {
    # md5: bfb5fec661318f07b0eca8520bb8c53f 
    # sha256:  92a1d459194d0bf86ff26103a9c92d64059e1caa9d98e1ed9002058a0da8f53d
    # sample: https://malwr.com/analysis/Mzc0MTU0MTQ1YjRlNGVhYzgzMmQ0MGQ3YWY1NWUzZDg/
    'seed': 9099,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
8: {
    # md5: a075610a69e196ab74f79508dbcf5eef
    # sha256: caa6e59e98c22a3f9159412a612ad170d2683640e1845afb6f533f279f5e6577
    # sample: https://malwr.com/analysis/OWJhZjQ5ZmE4YjY4NDFhYmFhNjIxZDcyYmFkYzlhYTM/
    'seed': 9047,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
9: {
    # md5: 6c3e68307d01e4340c83fac94f237237  
    'seed': 9133,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
10: {
    # md5:  17bf0d1776de896315cb2d63118f9667
    # sha256: 98d6ebd37c861beaf7420494aa8dfd43e4904145bac62c607965b3a8d92967c1
    # sample: https://malwr.com/analysis/NWJiNjJhZWU2YjU2NGRlYjg5Njk2ZGJlMzZkZDcxYWI/
    'seed': 9199,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    },
11: {
    # md5:  
    # sha256: 
    # sample: 
    'seed': 9190,
    'shift': 7,
    'tlds': ['ru', 'info', 'biz', 'click', 'su', 'work', 'pl', 'org', 'pw',
        'xyz']
    }
}

def ror32(v, s):
    v &= 0xFFFFFFFF
    return ((v >> s) | (v << (32-s))) & 0xFFFFFFFF

def rol32(v, s):
    v &= 0xFFFFFFFF
    return ((v << s) | (v >> (32-s))) & 0xFFFFFFFF

def dga(date, config_nr, domain_nr, add_tld=False): 
    c = config[config_nr]
    seed_shifted = rol32(c['seed'], 17)
    dnr_shifted = rol32(domain_nr, 21)

    k = 0
    year = date.year
    for _ in range(7):
        t_0 = ror32(0xB11924E1 * (year + k + 0x1BF5), c['shift']) & 0xFFFFFFFF
        t_1 = ((t_0 + 0x27100001) ^ k) & 0xFFFFFFFF
        t_2 = (ror32(0xB11924E1 * (t_1 + c['seed']), c['shift'])) & 0xFFFFFFFF
        t_3 = ((t_2 + 0x27100001) ^ t_1) & 0xFFFFFFFF
        t_4 = (ror32(0xB11924E1 * (date.day//2 + t_3), c['shift'])) & 0xFFFFFFFF
        t_5 = (0xD8EFFFFF - t_4 + t_3) & 0xFFFFFFFF
        t_6 = (ror32(0xB11924E1 * (date.month + t_5 - 0x65CAD), c['shift'])) & 0xFFFFFFFF
        t_7 = (t_5 + t_6 + 0x27100001) & 0xFFFFFFFF
        t_8 = (ror32(0xB11924E1 * (t_7 + seed_shifted + dnr_shifted), c['shift'])) & 0xFFFFFFFF
        k = ((t_8 + 0x27100001) ^ t_7) & 0xFFFFFFFF
        year += 1

    length = (k % 11) + 7
    domain = ""
    for i in range(length):
        k = (ror32(0xB11924E1*rol32(k, i), c['shift']) + 0x27100001) & 0xFFFFFFFF
        domain += chr(k % 25 + ord('a'))

    k = ror32(k*0xB11924E1, c['shift'])
    tlds = c['tlds']
    tld_i = ((k + 0x27100001) & 0xFFFFFFFF) % len(tlds)

    if add_tld:
        domain += '.' + tlds[tld_i]
    return domain

def generate_locky(nr, d=None, config=1):
    ret = []

    d = d if d else datetime.now()

    for i in range(nr):
        ret.append(dga(d, config, i))

    return ret

#include lockyv2
#lockyv2.generate_locky(100)

In [0]:
# Original: https://github.com/baderj/domain_generation_algorithms/blob/master/pykspa/improved/dga.py
import json
import argparse
from datetime import datetime
import time

import sys
import os

def get_sld(length, seed):
    sld = ""
    modulo = 541 * length + 4
    a = length * length
    for i in range(length):
        index = (a + (seed*((seed % 5) + (seed % 123456) +
            i*((seed & 1) + (seed % 4567))) & 0xFFFFFFFF))  % 26
        a += length;
        a &= 0xFFFFFFFFF
        sld += chr(ord('a') + index)
        seed += (((7837632 * seed * length) & 0xFFFFFFFF) + 82344) % modulo;
    return sld


def generate_pykspa(nr, date=None, set_nr=2, add_tld=False):
    if not date:
        date = datetime.now()
    path = "/content/drive/My Drive/dga_predict-master/dga_classifier/dga_generators/set1_seeds.json"
    fl = os.path.join(os.path.dirname(os.path.realpath(path)), "set{}_seeds.json".format(set_nr))
    with open(fl, "r") as r:
        seeds = json.load(r)
    dt = time.mktime(date.timetuple())
    days = 20 if set_nr == 1 else 1
    index = int(dt//(days*3600*24))
    if str(index) not in seeds:
        print("Sorry, {} is out of the time range I know".format(date))
        return
    seed = int(seeds.get(str(index), None), 16)
    original_seed = seed 

    ret = []
    for dga_nr in range(nr):
        # determine next seed
        s = seed % (dga_nr + 1)
        seed += (s + 1)
        
        # second level length
        length = (seed + dga_nr) % 7 + 6  

        # get second level domain
        second_level_domain = get_sld(length, seed)

        # get first level domain
        if add_tld:
            tlds = ['com', 'net', 'org', 'info', 'cc']
            top_level_domain = tlds[(seed & 3)]

            # concatenate and print domain
            domain = second_level_domain + '.' +  top_level_domain
        else:
            domain = second_level_domain
        ret.append(domain)

    return ret


#import pykspa
#import datetime
#pykspa.generate_domains(datetime.datetime.now(), 10000, 2)
#or
#pykspa.generate_domains(datetime.datetime.now(), 10000, 2)

In [0]:
# Ramdo DGA
# https://www.damballa.com/behind-ramdo-dga-domain-generation-algorithm/


def generate_ramdo1(seed_num, domain_iterator, length=0x10, add_tld=False):

    xor1 = 0
    sh1 = seed_num << 1
    domain_iterator += 1
    step1 = domain_iterator * sh1
    # step1b = domain_iterator * seed_num
    domain_iterator -= 1
    iter_seed = domain_iterator * seed_num
    imul_edx = iter_seed * 0x1a
    xor1 = step1 ^ imul_edx
    domain_length = 0
    dom = ''

    while domain_length < length:
        # xor1_divide = xor1 / 0x1a
        xor1_remainder = xor1 % 0x1a
        xo1_rem_20 = xor1_remainder + 0x20
        xo1_step2 = xo1_rem_20 ^ 0xa1
        dom_byte = 0x41 + (0xa1 ^ xo1_step2)
        dom += chr(dom_byte)
        imul_iter = domain_length * step1
        imul_result = domain_length * imul_iter
        imul_1a = 0x1a * imul_result
        xor2 = xor1 ^ imul_1a
        xor1 = xor1 + xor2
        domain_length += 1

    domain_iterator += 1

    if add_tld:
        return dom + ".org", domain_iterator
    return dom, domain_iterator

def generate_ramdo(num_domains, seed_num=5, length=0x10):
    domains = []
    domain_iterator = 0
    for i in range(num_domains):
        domain, domain_iterator = generate_ramdo1(seed_num, domain_iterator, length)
        domains.append(domain)

    return domains


# import ramdo
# ramdo.generate_domains(10, 5)

In [0]:
#DGA
def gen_malicious(num_per_dga=10000):
    """Generates num_per_dga of each DGA"""
    domains =[]
    labels =[]
    
#*****************************************************************************
    # We use some arbitrary seeds to create domains with banjori
    banjori_seeds = ['somestring', 'firetruck', 'bulldozer', 'airplane', 'racecar',
                     'apartment', 'laptop', 'laptopcomp', 'malwareisbad', 'crazytrain',
                     'thepolice', 'fivemonkeys', 'hockey', 'football', 'baseball',
                     'basketball', 'trackandfield', 'fieldhockey', 'softball', 'redferrari',
                     'blackcheverolet', 'yellowelcamino', 'blueporsche', 'redfordf150',
                     'purplebmw330i', 'subarulegacy', 'hondacivic', 'toyotaprius',
                     'sidewalk', 'pavement', 'stopsign', 'trafficlight', 'turnlane',
                     'passinglane', 'trafficjam', 'airport', 'runway', 'baggageclaim',
                     'passengerjet', 'delta1008', 'american765', 'united8765', 'southwest3456',
                     'albuquerque', 'sanfrancisco', 'sandiego', 'losangeles', 'newyork',
                     'atlanta', 'portland', 'seattle', 'washingtondc']

    segs_size = int(max(1, num_per_dga/len(banjori_seeds)))
    for banjori_seed in banjori_seeds:
        domains += generate_banjori(segs_size, banjori_seed)
    labels += ['banjori']*len(banjori_seeds)*20
#*******************************************************************
    # Create different length domains using cryptolocker
    crypto_lengths = range(8, 32)
    segs_size = int(max(1, num_per_dga/len(crypto_lengths)))
    for crypto_length in crypto_lengths:
        domains += generate_cryptolocker(segs_size, seed_num=random.randint(1, 1000000), length=crypto_length)
        labels += ['cryptolocker']*segs_size
 #*******************************************************************
    #dircrypt
    domains += generate_dircrypt(num_per_dga)
    labels += ['dircrypt']*num_per_dga
 
#*******************************************************************   
    #corebot
    domains += generate_corebot(num_per_dga)
    labels += ['corebot']*num_per_dga

#*******************************************************************
    # generate kraken and divide between configs
    kraken_to_gen = int(max(1, num_per_dga/2))
    domains += generate_kraken(kraken_to_gen, datetime(2016, 1, 1), 'a', 3)
    labels += ['kraken']*kraken_to_gen
    domains += generate_kraken(kraken_to_gen, datetime(2016, 1, 1), 'b', 3)
    labels += ['kraken']*kraken_to_gen
#*******************************************************************
    # generate locky and divide between configs
    locky_gen = int(max(1, num_per_dga/11))
    for i in range(1, 12):
        domains += generate_locky(locky_gen, config=i)
        labels += ['locky']*locky_gen

#*******************************************************************
    # Generate pyskpa domains
    domains += generate_pykspa(num_per_dga, datetime(2016, 1, 1))
    labels += ['pykspa']*num_per_dga

#*******************************************************************
  
    # Generate qakbot
    domains += generate_qakbot(num_per_dga, tlds=[])
    labels += ['qakbot']*num_per_dga
#*******************************************************************
    # ramdo divided over different lengths
    ramdo_lengths = range(8, 32)
    segs_size = int(max(1, num_per_dga/len(ramdo_lengths)))
    for rammdo_length in ramdo_lengths:
        domains += generate_ramdo(segs_size, seed_num=random.randint(1, 1000000), length=rammdo_length)
        labels += ['ramdo']*segs_size

#*******************************************************************

    # ramnit
    domains += generate_ramnit(num_per_dga, 0x123abc12)
    labels += ['ramnit']*num_per_dga
#*******************************************************************

    # simda
    simda_lengths = range(8, 32)
    segs_size = int(max(1, num_per_dga/len(simda_lengths)))
    for simda_length in range(len(simda_lengths)):
        domains += generate_simda(segs_size, length=simda_length, tld=None, base=random.randint(2, 2**32))
        labels += ['simda']*segs_size
    
    
    return domains, labels
#*******************************************************************

In [0]:
# Our ourput file containg all the training data

#DATA_FILE = 'traindata.pkl'

def gen_data(force=False):
    domains =[]
    labels =[]
    """Grab all data for train/test and save

    force:If true overwrpklite, else skip if file
          already exists
    """
    #if force or (not os.path.isfile(DATA_FILE)):
    domains, labels = gen_malicious(1000)
    #print("gen_malicious d,l",domains, labels)
    length = len(domains)
        # Get equal number of benign/malicious
    alx_dom, alx_lbl = get_alexa(length)
    for i in alx_dom:
      domains.append(i)
    for i in alx_lbl:
      labels.append(i)
    #print("get_alexa d,l",domains, labels)
   
    ziped = list(zip(labels, domains))
    return ziped

In [0]:
def get_data(force=False):
    """Returns data and labels"""
    return gen_data(force)

In [0]:
"""Train and test LSTM classifier"""

import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split


def build_lstm_model(max_features, maxlen):
    """Build LSTM model"""
    model = Sequential()
    model.add(Embedding(max_features, 128, input_length=maxlen))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop')

    return model

def lstm_run(max_epoch=25, nfolds=10, batch_size=128):
    """Run train/test on logistic regression model"""
    # Extract data and labels
    indata = get_data()
    X = [x[1] for x in indata]
    #print("x=",X)
    indata1 = get_data()
    labels = [x[0] for x in indata1]

    # Generate a dictionary of valid characters
    valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(X)))}

    max_features = len(valid_chars) + 1
    maxlen = np.max([len(x) for x in X])

    # Convert characters to int and pad
    X = [[valid_chars[y] for y in x] for x in X]
    X = sequence.pad_sequences(X, maxlen=maxlen)

    # Convert labels to 0-1
    y = [0 if x == 'benign' else 1 for x in labels]

    final_data = []

    for fold in range(nfolds):
        print("fold %u/%u" % (fold+1, nfolds))
        X_train, X_test, y_train, y_test, _, label_test = train_test_split(X, y, labels, test_size=0.2)

        print('Build model...')
        model = build_lstm_model(max_features, maxlen)

        print("Train...")
        X_train, X_holdout, y_train, y_holdout = train_test_split(X_train, y_train, test_size=0.05)
        best_iter = -1
        best_auc = 0.0
        out_data = {}

        for ep in range(max_epoch):
            model.fit(X_train, y_train, batch_size=None, epochs=1)

            t_probs = model.predict_proba(X_holdout)
            t_auc = sklearn.metrics.roc_auc_score(y_holdout, t_probs)

            print('Epoch %d: auc = %f (best=%f)' % (ep, t_auc, best_auc))

            if t_auc > best_auc:
                best_auc = t_auc
                best_iter = ep

                probs = model.predict_proba(X_test)

                out_data = {'y':y_test, 'labels': label_test, 'probs':probs, 'epochs': ep,
                            'confusion_matrix': sklearn.metrics.confusion_matrix(y_test, probs > .5)}

                print(sklearn.metrics.confusion_matrix(y_test, probs > .5))
            else:
                # No longer improving...break and calc statistics
                if (ep-best_iter) > 2:
                    break

        final_data.append(out_data)

    return final_data


In [0]:
def build_bigram_model(max_features):
    """Builds logistic regression model"""
    model = Sequential()
    model.add(Dense(1, input_dim=max_features, activation='sigmoid'))
    #model.Summary()
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score
def bigram_run(max_epoch=50, nfolds=10, batch_size=128):
    """Run train/test on logistic regression model"""
    indata = get_data()
    #print("indata=",tuple(indata))
    # Extract data and labels
    X = [x[1] for x in indata]
    #print("X=",X)
    indata1 = get_data()
    labels = [x[0] for x in indata1]
    #print("labels=",labels)

    # Convert labels to 0-1
    y = [0 if i == 'benign' else 1 for i in labels]
    #print("labels y=",y)
    #print("labels=",labels)
    # Create feature vectors
    print("vectorizing data")
    ngram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
    count_vec = ngram_vectorizer.fit_transform(X)
    #print("ngram_vectorizer.get_feature_names():", ngram_vectorizer.get_feature_names())
    #print("count_vec after fit_transform", count_vec)
    max_features = count_vec.shape[1]
    print("max_features after count_vec.shape:", max_features)
   
    
    final_data = []

    for fold in range(nfolds):
        print("fold %u/%u" % (fold+1, nfolds))
        X_train, X_test, y_train, y_test, _, label_test = train_test_split(count_vec, y, labels, test_size=0.2)
        #print("X_train, X_test, y_train, y_test, _, label_test = train_test_split(count_vec, y, labels, test_size=0.2)", X_train, X_test, y_train, y_test, _, label_test)
        print('Build model...')
        model = build_bigram_model(max_features)

        print("Train...")
        X_train, X_holdout, y_train, y_holdout = train_test_split(X_train, y_train, test_size=0.05)
        #print("train_test_split: X_train, X_holdout, y_train, y_holdout = train_test_split(X_train, y_train, test_size=0.05)", X_train, X_holdout, y_train, y_holdout)
        best_iter = -1
        best_auc = 0.0
        out_data = {}

        for ep in range(max_epoch):
            model.fit(X_train.todense(), y_train, batch_size=batch_size, epochs=1)

            t_probs = model.predict_proba(X_holdout.todense())
            print("y_holdout:", y_holdout)
            t_auc = roc_auc_score(y_holdout, t_probs)
            print('Epoch %d: auc = %f (best=%f)' % (ep, t_auc, best_auc))

            if t_auc > best_auc:
                best_auc = t_auc
                best_iter = ep

                probs = model.predict_proba(X_test.todense())

                out_data = {'y':y_test, 'labels': label_test, 'probs':probs, 'epochs': ep,
                            'confusion_matrix': sklearn.metrics.confusion_matrix(y_test, probs > .5)}

                print(sklearn.metrics.confusion_matrix(y_test, probs > .5))
            else:
                # No longer improving...break and calc statistics
                if (ep-best_iter) > 5:
                    break

        final_data.append(out_data)

    return final_data

In [0]:
def run_experiments(isbigram=True, islstm=True, nfolds=10):
    """Runs all experiments"""
    bigram_results = None
    lstm_results = None

    
    if isbigram:
        print("bigram_results")
        bigram_results = bigram_run(nfolds=nfolds)
    
    if islstm:
        print("lstm_results")
        lstm_results = lstm_run(nfolds=nfolds)

    return bigram_results, lstm_results

In [0]:
def create_figs(isbigram=True, islstm=True, nfolds=10, force=False):
    #Create figures
    # Generate results if needed
    #if force or (not os.path.isfile(RESULT_FILE)):
        #bigram_results, lstm_results = run_experiments(isbigram, islstm, nfolds)
    bigram_results, lstm_results = run_experiments(isbigram, islstm, nfolds)
    results = {'bigram': bigram_results, 'lstm': lstm_results}
    RESULT_FILE = 'results.pkl'
    with open(RESULT_FILE, 'ab') as data_f:
                pickle.dump(results, data_f)
        #pickle.dump(results, open(RESULT_FILE, 'w'))
    #else:
        #results = pickle.load(open(RESULT_FILE))

    # Extract and calculate bigram ROC
    if (results['bigram']):
        bigram_results = results['bigram']
        fpr = []
        tpr = []
        for bigram_result in bigram_results:
            t_fpr, t_tpr, _ = roc_curve(bigram_result['y'], bigram_result['probs'])
            fpr.append(t_fpr)
            tpr.append(t_tpr)
        bigram_binary_fpr, bigram_binary_tpr, bigram_binary_auc = calc_macro_roc(fpr, tpr)

    # xtract and calculate LSTM ROC
    if results['lstm']:
        lstm_results = results['lstm']
        fpr = []
        tpr = []
        for lstm_result in lstm_results:
            t_fpr, t_tpr, _ = roc_curve(lstm_result['y'], lstm_result['probs'])
            fpr.append(t_fpr)
            tpr.append(t_tpr)
        lstm_binary_fpr, lstm_binary_tpr, lstm_binary_auc = calc_macro_roc(fpr, tpr)

    # Save figure
    from matplotlib import pyplot as plt
    with plt.style.context('bmh'):
        plt.plot(lstm_binary_fpr, lstm_binary_tpr,
                label='LSTM (AUC = %.4f)' % (lstm_binary_auc, ), rasterized=True)
        plt.plot(bigram_binary_fpr, bigram_binary_tpr,
                label='Bigrams (AUC = %.4f)' % (bigram_binary_auc, ), rasterized=True)

        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=22)
        plt.ylabel('True Positive Rate', fontsize=22)
        plt.title('ROC - Binary Classification', fontsize=26)
        plt.legend(loc="lower right", fontsize=22)

        plt.tick_params(axis='both', labelsize=22)
        plt.savefig('results.png')

In [0]:
def calc_macro_roc(fpr, tpr):
    """Calcs macro ROC on log scale"""
    # Create log scale domain
    all_fpr = sorted(itertools.chain(*fpr))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(len(tpr)):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    return all_fpr, mean_tpr / len(tpr), auc(all_fpr, mean_tpr) / len(tpr)


In [0]:
if __name__ == "__main__":
    create_figs(nfolds=2) # Run with 1 to make it fast

bigram_results
vectorizing data
max_features after count_vec.shape: 1252
fold 1/2
Build model...





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train...



Epoch 1/1





16613/16613 [==============================] - 1s 46us/step - loss: 0.6126
y_holdout: [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 